# Introduction: Book Recommendation System

In this notebook, we will build a book recommendation system based on all the books articles on Wikipedia and neural network embeddings of books and links. This is a great look at the potential for neural networks to create meaningful embeddings of high dimensional data and one practical application of deep learning.

## Read in Data

The data is stored as json with line for every book. This data contains every single book article on Wikipedia which was parsed in the Downloading and Parsing Wikipedia Data Notebook.

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import json

books = []

with open('found_books_filtered.ndjson', 'r') as fin:
    # Append each line to the books
    books = [json.loads(l) for l in fin]

books_with_wikipedia = [book for book in books if 'Wikipedia:' in book[0]]
books = [book for book in books if 'Wikipedia:' not in book[0]]
print(f'Found {len(books)} books.')

Found 37020 books.


In [3]:
[book[0] for book in books_with_wikipedia]

['Wikipedia:Wikipedia Signpost/2014-06-25/Recent research',
 'Wikipedia:New pages patrol/Unpatrolled articles/December 2010',
 'Wikipedia:Templates for discussion/Log/2012 September 23',
 'Wikipedia:Articles for creation/Redirects/2012-10',
 'Wikipedia:Templates for discussion/Log/2012 October 4',
 'Wikipedia:Help desk/Archive 63',
 'Wikipedia:Teahouse/Questions/Archive 612',
 'Wikipedia:List of hoaxes on Wikipedia/La Croix du Sanguine Rouge',
 "Wikipedia:Administrators' noticeboard/Archive281",
 'Wikipedia:WikiProject Sailor Moon/Sailor Moon volume 01',
 'Wikipedia:Graphics Lab/Illustration workshop/Archive',
 'Wikipedia:New pages patrol/Unpatrolled articles/April 2011',
 'Wikipedia:Peer review/My Opposition (the Friedrich Kellner diary)/archive1',
 'Wikipedia:Manual of Style/Novels',
 'Wikipedia:Articles for creation/2007-07-04',
 'Wikipedia:WikiProject Warhammer 40,000/Templates',
 'Wikipedia:WikiProject Middle-earth/Templates',
 'Wikipedia:WikiProject Novels/The Mauritius Command t

Each book contains the title, the information from the `Infobox book` template, the internal wikipedia links, the external links, the date of last edit, and the number of characters in the article (a rough estimate of the length of the article).

In [4]:
books[15]

['Derech Mitzvosecha',
 {'author': 'Rabbi Menachem Mendel Schneersohn ( " Tzemach Tzedek " ), the third Rebbe of Chabad',
  'caption': 'Derech Mitzvosecha, 1912 edition',
  'genre': 'non-fiction',
  'image': 'Derech Mitzvosecha 2014-05-08 01-34.jpg',
  'image_size': '200px',
  'isbn': '0826655904',
  'language': 'Hebrew',
  'name': 'Derech Mitzvosecha (Sefer Hamitzvos)',
  'published': '7x10 Hardcover, 570pp, (Kehot Publication Society, Brooklyn New York)',
  'subject': 'Jewish mysticism, Chabad philosophy'},
 ['Mitzvos',
  'Menachem Mendel Schneersohn',
  'Rebbe',
  'Chabad',
  'Hasidic',
  'Chabad philosophy',
  'Tzitzit',
  'Tefillin',
  'Hillel the Elder',
  'Poltava',
  'Kehot Publication Society',
  'Eliyahu Touger',
  'Sichos in English',
  'Tanya',
  'Shneur Zalman of Liadi',
  'Category:Chabad-Lubavitch (Hasidic dynasty)',
  'Category:Books about Judaism',
  'Category:Chabad-Lubavitch texts'],
 ['http://www.books.google.com/books?id=HYLi3l9ylWwC',
  'http://www.store.kehotonli

# Map Books to Integers

First we want to create a mapping of books to integers. When we feed books into the neural network, we will have to represent them as numbers, so this mapping will let us keep track of the books.

In [5]:
book_index = {book[0]: idx for idx, book in enumerate(books)}
index_book = {idx: book for book, idx in book_index.items()}

book_index['War and Peace']
index_book[22460]

22460

'War and Peace'

# Number of Unique Wikilinks

First we can do a little data exploration. Let's find the number of unique Wikilinks (links to other Wikipedia articles) and the most common ones. To create a single list from a list of lists, we can use the `itertools` chain method.

In [6]:
from itertools import chain

wikilinks = list(chain(*[book[2] for book in books]))
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 311276 unique wikilinks.


How many of these are links to other books? 

In [7]:
wikilinks_other_books = [link for link in wikilinks if link in book_index.keys()]
print(f"There are {len(set(wikilinks_other_books))} unique links to other books.")

There are 17032 unique links to other books.


Let's take a look at the counts of each wikilink. We'll make a utility function that takes in a list and returns a sorted ordered dictionary of the counts of the items in the list.

In [8]:
from collections import Counter, OrderedDict

def count_items(l):
    """Return ordered dictionary of counts of objects in `l`"""
    # Create a counter object
    counts = Counter(l)
    
    # Sort by highest count first
    counts = sorted(counts.items(), key = lambda x: x[1], reverse = True)
    counts = OrderedDict(counts)
    
    return counts

In [9]:
wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

[('Hardcover', 7641),
 ('Paperback', 7482),
 ('Wikipedia:WikiProject Books', 6116),
 ('Wikipedia:WikiProject Novels', 6087),
 ('English language', 4307),
 ('The New York Times', 3960),
 ('United States', 3388),
 ('Science fiction', 3121),
 ('science fiction', 2661),
 ('Publishers Weekly', 2414)]

The most linked to pages are in fact not that surprising! One thing we should notice is that there are discrepancies in capitalization. We want to normalize across capitalization, so we'll lowercase all of the links and redo the counts.

In [10]:
wikilinks = [link.lower() for link in wikilinks]
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 297624 unique wikilinks.


In [25]:
wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:20]

[('paperback', 8976),
 ('hardcover', 8854),
 ('wikipedia:wikiproject books', 6116),
 ('wikipedia:wikiproject novels', 6088),
 ('science fiction', 5917),
 ('english language', 4371),
 ('the new york times', 3976),
 ('united states', 3391),
 ('novel', 3044),
 ('publishers weekly', 2414),
 ('the guardian', 2300),
 ('fantasy', 2094),
 ('kirkus reviews', 1741),
 ("children's literature", 1533),
 ('random house', 1523),
 ('harpercollins', 1480),
 ('fantasy novel', 1471),
 ('doctor who', 1323),
 ('category:american science fiction novels', 1289),
 ('non-fiction', 1186)]

That actually changes the rankings! This illustrates an important point: make sure to take a look at your data before modeling! 

In [12]:
wikilinks_other_books = [link.lower() for link in wikilinks_other_books]
wikilink_books_counts = count_items(wikilinks_other_books)
list(wikilink_books_counts.items())[:10]

[('the discontinuity guide', 148),
 ('the encyclopedia of science fiction', 145),
 ('dracula', 72),
 ('the dresden files', 70),
 ('the encyclopedia of fantasy', 66),
 ('encyclopædia britannica', 61),
 ('the wonderful wizard of oz', 61),
 ('nineteen eighty-four', 59),
 ("alice's adventures in wonderland", 54),
 ('don quixote', 52)]

Since there are so many unique wikilinks, I'm going to limit the list to wikilinks mentioned 5 or more times. Hopefully this reduces the noise that might come from wikilinks that only appear a few times.

In [13]:
links = [t[0] for t in wikilink_counts.items() if t[1] >= 5]
print(len(links))
links[-10:]

38554


['catherine ryan hyde',
 'life with billy (book)',
 'shan states',
 'burmese chronicles#ramanya',
 'northern ndebele people',
 'international journal of hindu studies',
 'charles henderson (character)',
 'anarchy: a journal of desire armed',
 'american museum of fly fishing',
 'key of perihelion']

The pound sign in the link name means that the link directs to a specific section on the Wikipedia page. Although we could normalize these to the title of the wikipedia article, we'll leave the pound sign in for now.

I'm also going to remove the __most popular__ wikilinks because these are not very informative. If a large number of books have the same wikilink, then the wikilink is not very distinguishing. 

In [26]:
to_remove = ['hardcover', 'paperback', 'wikipedia:wikiproject books', 'wikipedia:wikiproject novels']
for t in to_remove:
    links.remove(t)

[('paperback', 8976),
 ('hardcover', 8854),
 ('wikipedia:wikiproject books', 6116),
 ('wikipedia:wikiproject novels', 6088),
 ('science fiction', 5917),
 ('english language', 4371),
 ('the new york times', 3976),
 ('united states', 3391),
 ('novel', 3044),
 ('publishers weekly', 2414),
 ('the guardian', 2300),
 ('fantasy', 2094),
 ('kirkus reviews', 1741),
 ("children's literature", 1533),
 ('random house', 1523),
 ('harpercollins', 1480),
 ('fantasy novel', 1471),
 ('doctor who', 1323),
 ('category:american science fiction novels', 1289),
 ('non-fiction', 1186)]

## Wikilinks to Index

As with the books, we need to map the wikilinks to integers. We'll also create the reverse mapping.

In [27]:
link_index = {link: idx for idx, link in enumerate(links)}
index_link = {idx: link for link, idx in link_index.items()}

link_index['the new york times']
index_link[6]
len(link_index)

2

'the guardian'

38550

# Build a Training Set

In order for any machine learning model to learn, it needs a training set. We are going to treat this as a supervised learning problem: given an article, can the neural network determine whether or not a wikilink was present on that page? To do this, we need to create a dictionary of books and the associate links on their page. We already have this in the list of books, but now we need to change our data structure.

In [ ]:
# book_link_mapping = {book_index[book[0]]: [link_index[link.lower()] for link in book[2] if link.lower() in links] for book in books}

# book_link_mapping = {}

# for i, book in enumerate(books):
#     book_link_mapping[book_index[book[0]]] = []
#     for link in book[2]:
#         if link.lower() in links:
#             book_link_mapping[book_index[book[0]]].append(link_index[link.lower()])
    
#     if i % 100 == 0:
#         print(i, end = '\r')

For each book and each wikilink in the books articles, we'll add it to a list of pairs with the form (book_index, link_index). We also filter out the links that did not have at least 5 occurrences.

In [ ]:
pairs = []

for book in books:
    pairs.extend((book_index[book[0]], link_index[link.lower()]) for link in book[2] if link.lower() in links)
    
len(pairs), len(links), len(books)

We now have 805304 positive examples on which to train! Each pair represents one internal wikipedia link for one book. Let's look at a few examples.

In [ ]:
index_book[pairs[5000][0]], index_link[pairs[5000][1]]

In [ ]:
index_book[pairs[10000][0]], index_link[pairs[10000][1]]

## Split into Training and Validation

For training the neural network, we can split the pairs into a training set and a validation set. We want to make sure that none of the validation set is present in the training set.

In [ ]:
pairs_set = set(pairs)

validation_split = int(0.2 * len(pairs))

train_pairs = pairs[validation_split:]
train_set = set(train_pairs)

valid_pairs = pairs[:validation_split]
valid_pairs = [pair for pair in valid_pairs if pair not in train_set]

len(train_pairs), len(valid_pairs)

In [ ]:
x = Counter(pairs)
sorted(x.items(), key = lambda x: x[1], reverse = True)[:5]

In [23]:
index_book[13337], index_link[729]

("France's Songs of the Bards of the Tyne - 1850", 'joseph philip robson')

In [24]:
index_book[31899], index_link[44]

('The Early Stories: 1953–1975', 'the new yorker')

The more common wikilinks might be less useful. In a way, this could be similar to problem that creates a need for TF-IDF: the more common a word, the less representative it is of the sentence.

## Generator For Training Samples

We want to generate positive samples and negative samples. The positive samples are simple: pick one random pair from the `pairs` and assign it a 1. The negative samples are also fairly easy: pick one random link and one random book, make sure they are not in the `pairs`, and assign them a -1.

In [ ]:
import numpy as np
import random
random.seed(100)

def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0, accuracy = False):
    """Generate batches of samples for training"""
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    
    if accuracy:
        neg_label = 0
    else:
        neg_label = -1
    
    while True:
        # randomly choose positive examples
        for idx, (book_id, link_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (book_id, link_id, 1)
            
        idx = n_positive
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_book = random.randrange(len(books))
            random_link = random.randrange(len(links))
            
            # Check to make sure this is not a positive example
            if (random_book, random_link) not in pairs_set:
                batch[idx, :] = (random_book, random_link, neg_label)
                idx += 1
                
        np.random.shuffle(batch)
        yield {'book': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]

In [ ]:
next(generate_batch(pairs, n_positive = 2, negative_ratio = 2))

## Book Embedding Model

In [ ]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

In [ ]:
def book_embedding_model(embedding_size = 50, accuracy = False):
    book = Input(name = 'book', shape = [1])
    link = Input(name = 'link', shape = [1])
    
    book_embedding = Embedding(name = 'book_embedding',
                               input_dim = len(book_index),
                               output_dim = embedding_size)(book)
    
    link_embedding = Embedding(name = 'link_embedding',
                               input_dim = len(link_index),
                               output_dim = embedding_size)(link)
    
    book_embedding = Reshape(target_shape = [embedding_size])(book_embedding)
    link_embedding = Reshape(target_shape = [embedding_size])(link_embedding)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 1)([book_embedding, link_embedding])
    
    if accuracy:
        out = Dense(1, activation = 'sigmoid')(merged)
        model = Model(inputs = [book, link], outputs = out)
        model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    else:
#         merged = Dense(1, activation = None)(merged)
        model = Model(inputs = [book, link], outputs = merged)
        model.compile(optimizer = 'Adam', loss = 'mse')
    
    return model

model = book_embedding_model()
model.summary()

# Train Model

The next step is just to train the model. We can fit the model on the generator and also make a generator for validation. 

In [ ]:
len(pairs) / 512

In [ ]:
n_positive = 512

train_gen = generate_batch(train_pairs, n_positive, negative_ratio=5)
valid_gen = generate_batch(valid_pairs, n_positive, negative_ratio=5)

model.fit_generator(train_gen, epochs = 20, 
                    steps_per_epoch = len(pairs) // n_positive,
                    validation_data = valid_gen, 
                    validation_steps = 10,
                    verbose = 2)

In [ ]:
model.save('first_attempt.h5')

## Extract Embeddings

In [ ]:
book_layer = model.get_layer('book_embedding')
book_weights = book_layer.get_weights()[0]
book_weights.shape

In [ ]:
book_weights = book_weights / np.linalg.norm(book_weights, axis = 1).reshape((-1, 1))

In [ ]:
np.sum(np.square(book_weights[0]))

In [ ]:
def find_similar_books(book, book_weights, n = 10, least = False):
    dists = np.dot(book_weights, book_weights[book_index[book]])
    
    if least:
        closest = np.argsort(dists)[:n]
        
    else:
        closest = np.argsort(dists)[-n:]
    
    for c in reversed(closest):
        print('Book:', index_book[c], 'Distance:', dists[c])
    
    return closest, dists

In [ ]:
x, y = find_similar_books('War and Peace', book_weights)

In [ ]:
x, y = find_similar_books('War and Peace', book_weights, least = True)

In [ ]:
x, y = find_similar_books('The Autobiography of Benjamin Franklin', book_weights)

In [ ]:
_ = find_similar_books('A More Perfect Constitution', book_weights)

In [ ]:
def find_

In [ ]:
x[-10:], y[x[-10:]]

## Model For Accuracy

In [ ]:
model_acc = book_embedding_model(50, True)

train_gen_acc = generate_batch(train_pairs, n_positive=512, negative_ratio=1, accuracy = True)
valid_gen_acc = generate_batch(valid_pairs, n_positive=512, negative_ratio=1, accuracy = True)

In [ ]:
model_acc.fit_generator(train_gen_acc, epochs = 10, steps_per_epoch=1000,
                        validation_data = valid_gen_acc, 
                        validation_steps = 10, verbose = 0)

In [ ]:
book_layer_acc = model_acc.get_layer('book_embedding')
book_weights_acc = book_layer_acc.get_weights()[0]

book_weights_acc = book_weights_acc / np.linalg.norm(book_weights_acc, axis = 1).reshape((-1, 1))

book_weights_acc.shape

In [ ]:
x, y = find_similar_books('War and Peace', book_weights_acc)

In [ ]:
l1 = books[book_index['War and Peace']][2]

In [ ]:
l2 = books[book_index['Son of Spellsinger']][2]

In [ ]:
for l in l1:
    if l in l2:
        print(l)

In [ ]:
_ = find_similar_books('A Storm of Swords', book_weights_acc)

In [ ]:
model.save('first_attempt_laptop.h5')

## Make Predictions

Of course the fun part is making predictions with the model. We can feed it any book and get the closest and furthest away books in the embedding space. 